In [ ]:
CREATE TABLE IF NOT EXISTS DATA_SLV.DIM_CITIES (
    city VARCHAR,
    id_city VARCHAR,
    country VARCHAR,
    distance NUMBER(38, 3),
    latitude NUMBER(38, 8),
    localized_country_name VARCHAR,
    longitude NUMBER(38, 8),
    member_count NUMBER(38, 0),
    ranking NUMBER(38, 0),
    state VARCHAR, 
    zip NUMBER(38, 0) 
);

MERGE INTO DATA_SLV.DIM_CITIES AS d
USING (
  SELECT DISTINCT
    -- limpieza básica
    IFF(LOWER(TRIM(city))='not_found', NULL, TRIM(city))            AS city,
    NULLIF(TRIM(TO_VARCHAR(city_id)), '')                           AS city_id,       -- uniforma a texto
    IFF(LOWER(TRIM(country))='not_found', NULL, TRIM(country))      AS country,
    TRY_TO_NUMBER(distance, 38, 3)                                  AS distance,
    -- valida rangos pero mantiene NUMBER
    CASE WHEN latitude  BETWEEN -90  AND  90  THEN latitude  ELSE NULL END AS latitude,
    TRIM(localized_country_name)                                    AS localized_country_name,
    CASE WHEN longitude BETWEEN -180 AND 180 THEN longitude ELSE NULL END AS longitude,
    TRY_TO_NUMBER(member_count)                                     AS member_count,
    TRY_TO_NUMBER(ranking)                                          AS ranking,
    IFF(LOWER(TRIM(state))='not_found', NULL, TRIM(state))          AS state,
    TRY_TO_NUMBER(zip)                                              AS zip,

    -- clave natural flexible: prioriza city_id; si no, compuesta country|state|city|zip
    COALESCE(
      NULLIF(TRIM(TO_VARCHAR(city_id)), ''),
      CONCAT(
        '#',
        COALESCE(TRIM(IFF(LOWER(TRIM(country))='not_found', NULL, TRIM(country))), ''), '|',
        COALESCE(TRIM(IFF(LOWER(TRIM(state))  ='not_found', NULL, TRIM(state))),   ''), '|',
        COALESCE(TRIM(IFF(LOWER(TRIM(city))   ='not_found', NULL, TRIM(city))),    ''), '|',
        COALESCE(TO_VARCHAR(TRY_TO_NUMBER(zip)), '')
      )
    ) AS nk
  FROM MEETUP_DB.DATA_BRZ.CITIES
) AS s
ON COALESCE(
     NULLIF(TRIM(d.id_city), ''),
     CONCAT(
       '#',
       COALESCE(TRIM(d.country), ''), '|',
       COALESCE(TRIM(d.state),   ''), '|',
       COALESCE(TRIM(d.city),    ''), '|',
       COALESCE(TO_VARCHAR(d.zip), '')
     )
   ) = s.nk

WHEN MATCHED THEN UPDATE SET
  d.city                    = s.city,
  d.id_city                 = s.city_id,
  d.country                 = s.country,
  d.distance                = s.distance,
  d.latitude                = s.latitude,
  d.localized_country_name  = s.localized_country_name,
  d.longitude               = s.longitude,
  d.member_count            = s.member_count,
  d.ranking                 = s.ranking,
  d.state                   = s.state,
  d.zip                     = s.zip

WHEN NOT MATCHED THEN INSERT (
  city, id_city, country, distance, latitude, localized_country_name,
  longitude, member_count, ranking, state, zip
) VALUES (
  s.city, s.city_id, s.country, s.distance, s.latitude, s.localized_country_name,
  s.longitude, s.member_count, s.ranking, s.state, s.zip
);